# Train Distributed Model

In [ ]:
import tensorflow as tf
# Server 0:
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})

In [ ]:
server0 = tf.train.Server(cluster, job_name="local", task_index=0)

print(server0)

In [ ]:
server1 = tf.train.Server(cluster, job_name="local", task_index=1)

print(server1)

In [ ]:
import tensorflow as tf
import datetime
import numpy as np

n = 2
c1 = tf.Variable([])
c2 = tf.Variable([])

def matpow(M, n):
    if n < 1: 
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

# Assigns ops to the local worker by default.
#with tf.device(tf.train.replica_device_setter(
#        worker_device="/job:worker/task:%d" % FLAGS.task_index,
#        cluster=cluster)):



In [ ]:
with tf.device("/job:local/task:0/cpu:0"):
    A = tf.random_normal(shape=[10000, 10000])
    c1 = matpow(A,n)

with tf.device("/job:local/task:1/cpu:0"):
    B = tf.random_normal(shape=[10000, 10000])
    c2 = matpow(B,n)

with tf.Session("grpc://127.0.0.1:2222") as sess:
    sum = c1 + c2
    start_time = datetime.datetime.now()
    print(sess.run(sum))
    print("Multi node computation time: " + str(datetime.datetime.now() - start_time))

In [ ]:
import requests

url = 'http://169.254.169.254/computeMetadata/v1/instance/network-interfaces/0/access-configs/0/external-ip'
headers = {'Metadata-Flavor': 'Google'}

r = requests.get(url, headers=headers)

ip_address = r.text
print(ip_address)

In [ ]:
import requests
import json

ip_address = '100.96.1.1'

url = 'http://allocator.demo.pipeline.io/allocation/%s' % ip_address

r = requests.get(url)

allocation = r.text
allocation_json = json.loads(allocation)
print(allocation_json['index'])